<a href="https://colab.research.google.com/github/Nrmnzcn/COVID19/blob/master/2_Covid19_Turkey_Compare_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
pip install fbprophet

In [97]:
import numpy as np 
import pandas as pd 

from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from fbprophet.plot import add_changepoints_to_plot

import plotly.offline as py
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

from sklearn.metrics import mean_absolute_error

import itertools
from statsmodels.tsa.arima_model import ARIMA

from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor

import warnings
warnings.simplefilter('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [98]:
column = ["Date", "Total Test", "Total Cases", "Total Deaths", "Total Recovered", 
          "Daily Cases", "Daily Test", "Daily Deaths", "Daily Recovered", "Active Cases" ]

In [99]:
data = pd.read_csv("https://raw.githubusercontent.com/Nrmnzcn/COVID19/master/data/tr_data.csv?token=AKBXLYDUX73BAWSSN3IONDTAMDFYG")
data.Date = pd.to_datetime(data.Tarih)
data = data.fillna(0)
data.head()

,Tarih,Toplam Test,Toplam Vaka,Toplam Vefat,Toplam İyileşen,Günlük Hasta,Günlük Test,Günlük Vefat,Günlük İyileşen,Aktif Vaka
0,2021-03-27,37657127.0,3179115,30923.0,2939929.0,1281.0,223214.0,151.0,18892.0,208263.0
1,2021-03-26,37433913.0,3149094,30772.0,2921037.0,1253.0,220985.0,153.0,20208.0,197285.0
2,2021-03-25,37212928.0,3120013,30619.0,2900829.0,1210.0,222753.0,157.0,19186.0,188565.0
3,2021-03-24,36990175.0,3091282,30462.0,2881643.0,1142.0,221738.0,146.0,17761.0,179177.0
4,2021-03-23,36768437.0,3061520,30316.0,2863882.0,1006.0,211848.0,138.0,19201.0,167322.0


In [100]:
data.shape

(382, 10)

In [101]:
data.columns = column

case = data[["Date","Total Cases"]]
deaths = data[["Date","Total Deaths"]]
recoveries = data[["Date","Total Recovered"]]

In [102]:
case.columns = ['ds', 'y']
deaths.columns = ['ds', 'y']
recoveries.columns = ['ds', 'y']

In [103]:
f_date = pd.to_datetime('2020-03-11')
s_date = pd.to_datetime('2021-02-27')
e_date = pd.to_datetime('2021-03-27')
l_date = pd.to_datetime('2021-04-27')

In [104]:
# CASE

case_df = case.copy()
case_df.ds = pd.to_datetime(case_df.ds)

C_train = case_df.loc[(case_df['ds'] >= f_date) & (case_df['ds'] <= s_date)]
C_test = case_df.loc[(case_df['ds'] >= s_date) & (case_df['ds'] <= e_date)]
C_result = C_test.copy()

# DEATHS

deaths_df = deaths.copy()
deaths_df.ds = pd.to_datetime(deaths_df.ds)

D_train = deaths_df.loc[(deaths_df['ds'] >= f_date) & (deaths_df['ds'] <= s_date)]
D_test = deaths_df.loc[(deaths_df['ds'] >= s_date) & (deaths_df['ds'] <= e_date)]
D_result = D_test.copy()


# RECOVERIES

recoveries_df = recoveries.copy()
recoveries_df.ds = pd.to_datetime(recoveries_df.ds)

R_train = recoveries_df.loc[(recoveries_df['ds'] >= f_date) & (recoveries_df['ds'] <= s_date)]
R_test = recoveries_df.loc[(recoveries_df['ds'] >= s_date) & (recoveries_df['ds'] <= e_date)]
R_result = R_test.copy()

In [105]:
print(len(R_test))

29


In [106]:
Model_C = Prophet(changepoint_prior_scale=.5) 
Model_C .fit(C_train )

Model_D = Prophet(changepoint_prior_scale=.5) 
Model_D .fit(D_train )

Model_R = Prophet(changepoint_prior_scale=.5) 
Model_R .fit(R_train )

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [107]:
Future_C = Model_C.make_future_dataframe(periods=len(C_train))
Future_D = Model_D.make_future_dataframe(periods=len(D_train))
Future_R = Model_R.make_future_dataframe(periods=len(D_train))

Forecast_C = Model_C.predict(Future_C)
Forecast_D = Model_D.predict(Future_D)
Forecast_R = Model_R.predict(Future_R)

In [108]:
C_valitate = Forecast_C.loc[(Forecast_C['ds'] >= s_date) & (Forecast_C['ds'] <= e_date)]
C_result["PROPHET"] = C_valitate['yhat'].values

D_valitate = Forecast_D.loc[(Forecast_D['ds'] >= s_date) & (Forecast_D['ds'] <= e_date)]
D_result["PROPHET"] = D_valitate['yhat'].values

R_valitate = Forecast_R.loc[(Forecast_R['ds'] >= s_date) & (Forecast_R['ds'] <= e_date)]
R_result["PROPHET"] = R_valitate['yhat'].values

ARİMA


In [109]:
C_train = np.array(C_train.y, dtype=np.float64)
C_test = np.array(C_test.y, dtype=np.float64)

D_train = np.array(D_train.y, dtype=np.float64)
D_test = np.array(D_test.y, dtype=np.float64)

R_train = np.array(R_train.y, dtype=np.float64)
R_test = np.array(R_test.y, dtype=np.float64)

In [110]:
p=d=q=range(0,4)
a=99999
pdq=list(itertools.product(p,d,q))
    
#Determining the best parameters
for var in pdq:
    try:
        model = ARIMA(C_train, order=var)
        result = model.fit()

        if (result.aic<=a) :
            a=result.aic
            param=var
    except:
        continue
        

model = ARIMA(C_train, order=param)
result = model.fit()
C_prediction=result.forecast(steps=len(C_test))[0]
C_result['ARIMA'] = C_prediction

print(result.summary()) 

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  352
Model:                 ARIMA(0, 2, 1)   Log Likelihood               -4392.523
Method:                       css-mle   S.D. of innovations          63037.690
Date:                Sun, 28 Mar 2021   AIC                           8791.047
Time:                        19:32:53   BIC                           8802.638
Sample:                             2   HQIC                          8795.659
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         61.5261     32.925      1.869      0.063      -3.006     126.059
ma.L1.D2.y    -1.0000      0.010   -101.826      0.000      -1.019      -0.981
                                    Roots           

In [111]:
p=d=q=range(0,4)
a=99999
pdq=list(itertools.product(p,d,q))
    
#Determining the best parameters
for var in pdq:
    try:
        model = ARIMA(D_train, order=var)
        result = model.fit()

        if (result.aic<=a) :
            a=result.aic
            param=var
    except:
        continue
        

model = ARIMA(D_train, order=param)
result = model.fit()
D_prediction=result.forecast(steps=len(C_test))[0]
D_result['ARIMA'] = D_prediction
print(result.summary()) 

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  352
Model:                 ARIMA(1, 2, 1)   Log Likelihood               -1033.550
Method:                       css-mle   S.D. of innovations              4.557
Date:                Sun, 28 Mar 2021   AIC                           2075.099
Time:                        19:33:03   BIC                           2090.554
Sample:                             2   HQIC                          2081.250
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1705      0.752      0.227      0.821      -1.302       1.643
ar.L1.D2.y     0.9382      0.031     30.090      0.000       0.877       0.999
ma.L1.D2.y    -0.8029      0.053    -15.109      0.0

In [112]:
p=d=q=range(0,4)
a=99999
pdq=list(itertools.product(p,d,q))
    
#Determining the best parameters
for var in pdq:
    try:
        model = ARIMA(R_train, order=var)
        result = model.fit()

        if (result.aic<=a) :
            a=result.aic
            param=var
    except:
        continue
        

model = ARIMA(R_train, order=param)
result = model.fit()
R_prediction=result.forecast(steps=len(C_test))[0]
R_result['ARIMA'] = R_prediction
print(result.summary()) 

                             ARIMA Model Results                              
Dep. Variable:                   D2.y   No. Observations:                  352
Model:                 ARIMA(0, 2, 1)   Log Likelihood               -4372.635
Method:                       css-mle   S.D. of innovations          59574.771
Date:                Sun, 28 Mar 2021   AIC                           8751.269
Time:                        19:33:15   BIC                           8762.860
Sample:                             2   HQIC                          8755.882
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         61.8811     31.117      1.989      0.048       0.894     122.868
ma.L1.D2.y    -1.0000      0.010    -99.241      0.000      -1.020      -0.980
                                    Roots           

**DECISION TREE REGRESSION**


In [113]:
def preprocessing(X, method):

    # scale, return[0-1]
    if method == 'MinMaxScaler':
        X_new = MinMaxScaler().fit_transform(X)

    return X_new

In [114]:
C_data = data[["Total Cases"]]
C_data = C_data.rename(columns={"Total Cases": "t"})

C_data["t-1"] = C_data["t"].shift(periods=1, fill_value=0)
C_data["t-2"] = C_data["t"].shift(periods=2, fill_value=0)
C_data["t-3"] = C_data["t"].shift(periods=3, fill_value=0)
C_data["t-4"] = C_data["t"].shift(periods=4, fill_value=0)
C_data["t-5"] = C_data["t"].shift(periods=5, fill_value=0)
C_data["t-6"] = C_data["t"].shift(periods=6, fill_value=0)
C_data["t-7"] = C_data["t"].shift(periods=7, fill_value=0)

C_data = C_data.replace([np.inf, -np.inf], np.nan)
C_data = C_data.dropna()

c_train = C_data.iloc[0:353]
c_test = C_data.iloc[353:]

print(len(c_test))

columns = ["t-1","t-2","t-3","t-4","t-5","t-6","t-7"]

X_c_train = c_train[columns].values
y_c_train = c_train["t"].values

X_c_test = c_test[columns].values
y_c_test = c_test["t"].values


X_c_train = preprocessing(X_c_train, 'MinMaxScaler')
X_c_test = preprocessing(X_c_test, 'MinMaxScaler')

dt = DecisionTreeRegressor()
dt.fit(X_c_train, y_c_train)
pred = dt.predict(X_c_test)
C_result['DT'] = pred

29


In [115]:
D_data = data[["Total Deaths"]]
D_data = D_data.rename(columns={"Total Deaths": "t"})

D_data["t-1"] = D_data["t"].shift(periods=1, fill_value=0)
D_data["t-2"] = D_data["t"].shift(periods=2, fill_value=0)
D_data["t-3"] = D_data["t"].shift(periods=3, fill_value=0)
D_data["t-4"] = D_data["t"].shift(periods=4, fill_value=0)
D_data["t-5"] = D_data["t"].shift(periods=5, fill_value=0)
D_data["t-6"] = D_data["t"].shift(periods=6, fill_value=0)
D_data["t-7"] = D_data["t"].shift(periods=7, fill_value=0)

D_data = D_data.replace([np.inf, -np.inf], np.nan)
D_data = D_data.dropna()

d_train = D_data.iloc[0:353]
d_test = D_data.iloc[353:]

columns = ["t-1","t-2","t-3","t-4","t-5","t-6","t-7"]

X_d_train = d_train[columns].values
y_d_train = d_train["t"].values

X_d_test = d_test[columns].values
y_d_test = d_test["t"].values


X_d_train = preprocessing(X_d_train, 'MinMaxScaler')
X_d_test = preprocessing(X_d_test, 'MinMaxScaler')

dt = DecisionTreeRegressor()
dt.fit(X_d_train, y_d_train)
pred = dt.predict(X_d_test)
D_result['DT'] = pred


In [116]:
R_data = data[["Total Recovered"]]
R_data = R_data.rename(columns={"Total Recovered": "t"})

R_data["t-1"] = R_data["t"].shift(periods=1, fill_value=0)
R_data["t-2"] = R_data["t"].shift(periods=2, fill_value=0)
R_data["t-3"] = R_data["t"].shift(periods=3, fill_value=0)
R_data["t-4"] = R_data["t"].shift(periods=4, fill_value=0)
R_data["t-5"] = R_data["t"].shift(periods=5, fill_value=0)
R_data["t-6"] = R_data["t"].shift(periods=6, fill_value=0)
R_data["t-7"] = R_data["t"].shift(periods=7, fill_value=0)

R_data = R_data.replace([np.inf, -np.inf], np.nan)
R_data = R_data.dropna()

r_train = R_data.iloc[0:353]
r_test = R_data.iloc[353:]

columns = ["t-1","t-2","t-3","t-4","t-5","t-6","t-7"]

X_r_train = r_train[columns].values
y_r_train = r_train["t"].values

X_r_test = r_test[columns].values
y_r_test = r_test["t"].values


X_r_train = preprocessing(X_r_train, 'MinMaxScaler')
X_r_test = preprocessing(X_r_test, 'MinMaxScaler')

dt = DecisionTreeRegressor()
dt.fit(X_r_train, y_r_train)
pred = dt.predict(X_r_test)
R_result['DT'] = pred

**RESULT**


In [117]:
print(" ********** CASE **********")
print("#### PROPHET ####")
MAPE = np.mean(abs((C_result['y'] - C_result['PROPHET']) / C_result['y'])) * 100
MAE = mean_absolute_error(C_result['y'], C_result['PROPHET'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### ARIMA ####")
MAPE = np.mean(abs((C_result['y'] - C_result['ARIMA']) / C_result['y'])) * 100
MAE = mean_absolute_error(C_result['y'], C_result['ARIMA'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### DT ####")
MAPE = np.mean(abs((C_result['y'] - C_result['DT']) / C_result['y'])) * 100
MAE = mean_absolute_error(C_result['y'], C_result['DT'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("\n")


print(" ********** DEATHS **********")
print("#### PROPHET ####")
MAPE = np.mean(abs((D_result['y'] - D_result['PROPHET']) / D_result['y'])) * 100
MAE = mean_absolute_error(D_result['y'], D_result['PROPHET'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### ARIMA ####")
MAPE = np.mean(abs((D_result['y'] - D_result['ARIMA']) / D_result['y'])) * 100
MAE = mean_absolute_error(D_result['y'], D_result['ARIMA'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### DT ####")
MAPE = np.mean(abs((D_result['y'] - D_result['DT']) / D_result['y'])) * 100
MAE = mean_absolute_error(D_result['y'], D_result['DT'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("\n")

print(" ********** RECOVERIES **********")
print("#### PROPHET ####")
MAPE = np.mean(abs((R_result['y'] - R_result['PROPHET']) / R_result['y'])) * 100
MAE = mean_absolute_error(R_result['y'], R_result['PROPHET'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### ARIMA ####")
MAPE = np.mean(abs((R_result['y'] - R_result['ARIMA']) / R_result['y'])) * 100
MAE = mean_absolute_error(R_result['y'], R_result['ARIMA'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("#### DT ####")
MAPE = np.mean(abs((R_result['y'] - R_result['DT']) / R_result['y'])) * 100
MAE = mean_absolute_error(R_result['y'], R_result['DT'])
print("MAE : ", MAE)
print("MAPE : ", MAPE)
print("\n")

 ********** CASE **********
#### PROPHET ####
MAE :  182773.8142680241
MAPE :  6.195643234739542
#### ARIMA ####
MAE :  2831111.2858168576
MAPE :  97.9418797745423
#### DT ####
MAE :  1776158.7241379311
MAPE :  61.94588795292606


 ********** DEATHS **********
#### PROPHET ####
MAE :  2547.0145895088276
MAPE :  8.755549511147672
#### ARIMA ####
MAE :  29441.44462391471
MAPE :  99.76696986400744
#### DT ####
MAE :  13931.827586206897
MAPE :  47.23563984370306


 ********** RECOVERIES **********
#### PROPHET ####
MAE :  171437.9494474253
MAPE :  6.3295525292608925
#### ARIMA ####
MAE :  2650813.0211781696
MAPE :  97.5775103132418
#### DT ####
MAE :  1169487.9310344828
MAPE :  42.444399709273576


